In [1]:
import numpy as np
import pandas as pd
from PIL import Image,ImageEnhance
import os
from osgeo import osr, gdal,gdal_array
from fractions import gcd
import math
import pathlib

In [2]:
output_dir = "data/tiles"
raw_file_location = "../data/digitalglobe/Escondido2014_utm_forcep5_clip.tif"
raw_file_name = "Escondido2014_utm_forcep5_clip"
#raw_file_location = "../data/planet_imagery/1154314_2014-07-23_RE2_3A_Analytic_clip.tif"
#raw_file_name = "1154314_2014-07-23_RE2_3A_Analytic_clip"
#raw_file_location = "l8b30f30clip.tif"
#raw_file_name = "l8b30f30clip"

# red, green, blue bands
color_band_order = [0,1,2]
#color_band_order = [4,3,2]

# square size
squaresize = 10

In [3]:
ds = gdal.Open(raw_file_location, gdal.GA_ReadOnly)
# DataType is a property of the individual raster bands
image_datatype = ds.GetRasterBand(1).DataType

# Allocate our array, but in a more efficient way
image_correct = np.zeros((ds.RasterYSize, ds.RasterXSize, ds.RasterCount),
                 dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))

# Loop over all bands in dataset
#for b in range(ds.RasterCount):
for b in range(ds.RasterCount):
    # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
    band = ds.GetRasterBand(b + 1)
    
    # Read in the band's data into the third dimension of our array
    image_correct[:, :, b] = band.ReadAsArray()

# Order the bands into red, green, blue
image_correct = image_correct[:, :, color_band_order]

In [4]:
cols = ds.RasterXSize
rows = ds.RasterYSize

transform = ds.GetGeoTransform()
minx = transform[0]
maxx = transform[0] + cols * transform[1] + rows * transform[2]

miny = transform[3] + cols * transform[4] + rows * transform[5]
maxy = transform[3]

width = maxx - minx
height = maxy - miny

xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

print (xOrigin, yOrigin)

480894.4629851412 3667501.770068448


In [5]:
m = int(rows/squaresize)
n = int(cols/squaresize)

print (rows,cols)
print (m,n)

29045 19200
2904 1920


In [7]:

driver = gdal.GetDriverByName('GTiff')
pathlib.Path(os.path.join(output_dir, raw_file_name, 'geotiff')).mkdir(parents=True, exist_ok=True) 

tile_num = 0

for j in range(m, 0, -1):
    for i in range(0, n):
        ulx = minx + (width/n) * i 
        uly = miny + (height/m) * j 
        i1 = int((ulx - xOrigin) / pixelWidth)
        j1 = int((yOrigin - uly)  / pixelHeight)
        new_x = xOrigin + i1*pixelWidth
        new_y = yOrigin - j1*pixelHeight
        new_transform = (new_x, transform[1], transform[2], new_y, transform[4], transform[5])
        
        output_file_base = raw_file_name + str(tile_num) + ".tif"
        output_file = os.path.join(output_dir, raw_file_name, 'geotiff', output_file_base)
        
        #print('a',str((m-j)*squaresize),str((m-j)*squaresize+squaresize),str(i*squaresize),str(i*squaresize+squaresize))
        #print(image_correct[(m-j)*squaresize:(m-j)*squaresize+squaresize, i*squaresize:i*squaresize+squaresize, :].shape)

        actual_shape = image_correct[(m-j)*squaresize:(m-j)*squaresize+squaresize, i*squaresize:i*squaresize+squaresize, :].shape
        
        dst_ds = driver.Create(output_file,
                               actual_shape[1],
                               actual_shape[0],
                               3,
                               gdal.GDT_Float32, options = [ 'PHOTOMETRIC=RGB' ])

        #writting output raster
        #dst_ds.GetRasterBand(1).WriteArray( data )
        for b in range(3):
            dst_ds.GetRasterBand(b + 1).WriteArray(image_correct[(m-j)*squaresize:(m-j)*squaresize+squaresize, i*squaresize:i*squaresize+squaresize, b])

        tif_metadata = {
            "minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy)
        }
        dst_ds.SetMetadata(tif_metadata)

        #setting extension of output raster
        # top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
        dst_ds.SetGeoTransform(new_transform)

        wkt = ds.GetProjection()

        # setting spatial reference of output raster
        srs = osr.SpatialReference()
        srs.ImportFromWkt(wkt)
        dst_ds.SetProjection( srs.ExportToWkt() )

        #Close output raster dataset
        dst_ds = None
        tile_num += 1

KeyboardInterrupt: 

In [8]:
list_patches = []
pathlib.Path(os.path.join(output_dir, raw_file_name, 'jpg')).mkdir(parents=True, exist_ok=True) 

for j in range(m, 0, -1):
    for i in range(0, n):
        list_patches.append(image_correct[(m-j)*squaresize:(m-j)*squaresize+squaresize, i*squaresize:i*squaresize+squaresize, :])
            
for x in range(len(list_patches)):
    blue = Image.fromarray(list_patches[x][:,:,2], "L")
    green = Image.fromarray(list_patches[x][:,:,1], "L")
    red = Image.fromarray(list_patches[x][:,:,0], "L")
    out = Image.merge("RGB", (red, green, blue))
    out.save(os.path.join(output_dir, raw_file_name, 'jpg', raw_file_name +str(x)+".jpg"))

KeyboardInterrupt: 